## Step 5: Multi-Node Performance Analysis

In this notebook, we will learn how to use Nsight Systems in a multi-node performance analysis.

## 5.1 Data Collection
Nsight Systems is not aware of any cluster schedulers such as Slurm. Thus, `nsys profile` should be put after the scheduler's run command, usually directly before the application, e.g.

`srun <srun args> nsys profile <nsys args> -o report_name.%q{SLURM_PROCID} your_application`

This will result in one report file per process or rank.

When data is collected for an increasing number of processes or ranks, it is reasonable to limit the recorded data. Otherwise the Nsight Systems GUI or the recipe analysis might not be able to handle the sheer amount of data. There are several ways to limit data recording:

- Capture ranges (`--capture-range`) are used to limit the interval in which data is collected. Capture ranges can be triggered via the CUDA profiler API and NVTX events.
- Set a collection duration (`--duration`).
- Delay data recording (`--delay`), e.g. to skip the initialization phase.
- NVTX domain filtering can include or exclude events from an NVTX domain (`--nvtx-domain-include` and `--nvtx-domain-exclude`).
- Record only a set of ranks or just a single one, e.g. with helper script

```bash
    #/bin/bash
    if [ $SLURM_LOCALID -eq 0 ]; then
      nsys profile "$@"
    else
      "$@"
    fi
```

## 5.2 Case Study: MLPerf

For this case study, we are using the MLPerf DeepCAM benchmark, which trains a deep learning segmentation model for identifying extreme weather phenomena in climate simulation data.
MLPerf DeepCAM uses DALI (NVIDIA Data Loading Library) to load the training data and NCCL (NVIDIA Collective Communication Library) to exchange data between GPUs.

Since we do not have access to a multi-node cluster in this lab, the reports have been precollected in the folder *reports/precollected/mlperf*, for an initial run and after an optimization step.
The profiling runs were executed on two nodes of EOS, an NVIDIA DGX SuperPOD cluster with 8 H100 GPUs per node.
The following commands have been used to run the benchmark code:
```bash
    # Apply system-wide options only to one rank per node
    if [ $SLURM_LOCALID -eq 0 ]; then
        nsys_flags_local0="--nic-metrics=true --storage-metrics=--storage-devices=all"
    fi

    srun -n 16 <more srun arguments> \
      nsys profile ${nsys_flags_local0} --trace=cuda,nvtx,osrt \
        --cuda-graph-trace=node \
        --capture-range=cudaProfilerApi --kill=none \
        -o ${OUTPUT_DIR}/mlperf_...${SLURM_PROCID} -f true \
        /usr/bin/python ./train.py --capture_range_start 11 --capture_range_stop 29 <more training parameters>
```

The command runs 16 ranks/processes across the two nodes.
The `--cuda-graph-trace=node` instructs to record at CUDA kernel granularity, which enables us to distinguish between NCCL and CUDA compute kernels. Otherwise, only the begin and end of the graph execution is recorded.
The `--capture-range=cudaProfilerApi` flag limits the recording to a specific range. The application code has to call the CUDA profiler APIs to make this work.
The `kill=none` flag instructs Nsight Systems to not send a signal to the application, when the profiling session ends.
System-wide Nsight Systems CLI flags, e.g. for network interface card (NIC) and storage metrics sampling, are only set for the node-local rank 0 to avoid collecting redundant data.

## 5.3 Report Inspection

The report files of the initial run are located in the folder *reports/precollected/mlperf/initial_run*.
It contains 16 report files, one for each rank.
Let's take a look at the report of rank 0 in the Nsight Systems GUI to get an idea on the code execution.
The file name is *mlperf2_bs8_nvme-posix_t1_steps11-29.00.nsys-rep*.

<center><img src=images/step5/mlperf_initial_run_nsys_timeline.png width=95%></center>

Some observations:
- Continuously reading (training data) with a throughput of about 14 GB/s.
- Lots of processes/threads are often in *pthread_cond_wait* state.
- One process is performing the file reads via DALI, which results in many `fopen` and `fread` operations.
- Step duration increases over time from about 100ms to 220ms. We ignore the first step (11) at the beginning of our capture range, since it likely includes some profiling startup overhead.
- `ncclAllReduce` duration varies quite a bit in each step.
- `ncclDevKernel_AllReduce_Sum_f32_RING` is the kernel with the highest execution percentage (about 24%).
- GPU idle time before the NCCL kernel `ncclDevKernel_AllReduce_Sum_f32_RING` increases over time.

Given these observations, the NCCL allreduce operation seems to wait for training data to be loaded from disk.

So far, we have only looked at a single report file, representing the execution of only a single rank and GPU.
There are several Nsight Systems recipes that can help us better understand the overall performance and identify inefficiencies in this multi-node program run.
To analyze the IO behavior over all disks and nodes, there is the **storage_util_map** recipe.
For applications that use NCCL, the **nccl_gpu_time_util_map** provides an overview on the temporal GPU utilization by NCCL and compute kernels as well as their overlap.

Since NCCL kernels are dominating the GPU usage on rank 0, let us investigate whether this is the case for all ranks.
Execute the following code cell to run the *NCCL GPU Time Utilization Heatmap* recipe on the reports of the initial MLPerf run.

In [3]:
!nsys recipe nccl_gpu_time_util_map \
--output reports/precollected/mlperf/initial_run/results_nccl_gpu_util_map \
--force-overwrite \
--bins 100 \
--log-level=error \
--input reports/precollected/mlperf/initial_run

usage: nccl_gpu_time_util_map [-h] [--output OUTPUT] [--force-overwrite]
                              --input INPUT [INPUT ...] [--bins BINS]
                              [--disable-alignment]
                              [--filter-time [start_time]/[end_time] |
                              --filter-nvtx range[@domain][/index]]
                              [--mode {none,concurrent,dask-futures}]
nccl_gpu_time_util_map: error: argument --input: reports/precollected/mlperf/initial_run does not exist.


After the recipe execution is finished, we can open the Jupyter notebook [reports/precollected/mlperf/initial_run/results_nccl_gpu_util_map/heatmap.ipynb](reports/precollected/mlperf/initial_run/results_nccl_gpu_util_map/heatmap.ipynb) and run all kernels.

Screenshots and observations:

<center><img src=images/step5/mlperf_initial_run_nccl_recipe_util_sum.png width=95%></center>
<center><img src=images/step5/mlperf_initial_run_nccl_recipe_heatmap_util.png width=90%></center>

The NCCL GPU *Utilization Summary* graph shows *NCCL* and *Compute* kernels alternating with no overlap.
The *Overlap* heatmap confirms the latter.
The *Utilization All* heatmap shows the overall GPU utilization per rank.

In general, the exection pattern represented by the heatmaps has some irregularities, which indicates inefficiencies.
A regular pattern is usually also better in terms of performance.
For parallel applications running in SPMD or a lock-step-like mode, time-aligned execution phases are ideal.
Otherwise, it is likely that synchronization between the ranks will cause waiting times.

## 5.4 Optimize Data Loading

A fundamental bottleneck in the initial execution is data loading from disks, which cannot cover the processing capacities of the GPUs.
Different data loading times ultimately also result in the irregular execution pattern.
One if the issues is that the kernel caches the data reading, which is not efficient for the large amount of data that is read only once. Using *O_DIRECT* for file reading improves the situation. Furthermore, it can be beneficial to use more threads for data loading. Hence, we used *O_DIRECT* and four instead of one thread for loading the training data in another execution of our MLPerf benchmark.

<div class="alert alert-block alert-info">
<b> Exercise</b>

<p>Execute the <i>NCCL GPU Time Utilization Heatmap</i> recipe for the profiles in <i>reports/precollected/mlperf/optimized_run</i>.
<br>What does the pattern look like?
<br>How is the GPU utilized by NCCL and compute kernels?</p>

<p>Inspect the report of rank 0 <i>mlperf2_bs8_nvme-odirect_t4_steps11-29.00.nsys-rep</i> using the Nsight Systems GUI for more in-depth analyses.
<br>How is the IO data reading throughput?
<br>What about the step duration?
<br>What happend to the DALI row/ranges?</p>

<p> You can use the following empty code cell to execute the recipe.</p>
</div>

In [ ]:
# Execute the NCCL GPU Time Utilization Heatmap recipe.
!nsys recipe nccl_gpu_time_util_map \
--output reports/precollected/mlperf/optimized_run/results_nccl_gpu_util_map \
--force-overwrite \
--bins 100 \
--log-level=error \
--input reports/precollected/mlperf/optimized_run

usage: nccl_gpu_time_util_map [-h] [--output OUTPUT] [--force-overwrite]
                              --input INPUT [INPUT ...] [--bins BINS]
                              [--disable-alignment]
                              [--filter-time [start_time]/[end_time] |
                              --filter-nvtx range[@domain][/index]]
                              [--mode {none,concurrent,dask-futures}]
nccl_gpu_time_util_map: error: argument --input: /dli/task/reports/precollected/mlperf/optimized_run does not exist.


Open the **NCCL GPU utilization** notebook [results_nccl_gpu_util_map/heatmap.ipynb](reports/precollected/mlperf/optimized_run/results_nccl_gpu_util_map/heatmap.ipynb) and run all kernels.

<center><img src=images/step5/mlperf_optimized_run_nccl_recipe_util_sum.png width=95%></center>
<center><img src=images/step5/mlperf_optimized_run_nccl_recipe_heatmap_util.png width=90%></center>

Observations:
- NCCL communication kernels are much shorter, but they still do not overlap with GPU computation.
- The execution pattern is much more aligned across the ranks.

The following screenshot shows the Nsight Systems GUI for the report of rank 0 *mlperf2_bs8_nvme-odirect_t4_steps11-29.00.nsys-rep*.

<center><img src=images/step5/mlperf_optimized_run_nsys_timeline.png width=95%></center>

The Nsight Systems timeline visualization of rank 0 confirms the observations from the recipe results.
It also shows that the step duration is almost constant now and only about half of the execution time than before.

Data is not read continuously any more, but in intervals with about three times the throughput than before.
Therefore, data reading is likely not the bottleneck any more. The *storage utilization* recipe would confirm this.
Feel free to try it on both, the initial and the optimized run and compare the results.

Still having GPU gaps (about 1ms) and NCCL device kernels that do not overlap with GPU computation indicate that there is optimization potential from a high-level execution perspective.

### 5.5 Summary

<div class="alert alert-block alert-success">
    <p>We investigated a bottleneck in an MLPerf multi-node execution using the Nsight Systems timeline and two Nsight Systems recipes.</p>
    <p>
        NCCL communication is performed via CUDA kernels and shown as compute kernels in the Nsight Systems timeline.<br>
        The NCCL recipes can help to expose overlap of NCCL communication and GPU computation.
    </p>
    <p>Consider running recipes on the reports of a parallel program to get an overview on the parallel execution before examining individual reports in the Nsight Systems GUI.</p>
</div>

Please click [here](summary.ipynb) to move to the end of the Nsight Systems lab.